In [53]:
import pandas as pd
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [54]:
last_fy = date.today().year - 1
prev_prev_fy = date.today().year - 2

In [55]:
print(last_fy)

2023


In [56]:
modcols = {
    'Unnamed: 0' : 'metric_type',
    'Unnamed: 1' : f"{last_fy} Totals",
    'Unnamed: 2' : f"Perc of {last_fy}",
    'Unnamed: 3' : f"{prev_prev_fy} Totals",
    'Unnamed: 4' : f"Perc of {prev_prev_fy}"
}

In [57]:
df = pd.read_csv("data/2yrcompare/bwp_2023_compare.csv", index_col = False)

In [58]:
# df.head()

In [59]:
df = df.rename(columns = modcols)

In [60]:
df = df.dropna(axis=0)

In [61]:
paranthesis1 = df[f"{last_fy} Totals"].str.contains(r'\(|\)')
paranthesis2 = df[f"{prev_prev_fy} Totals"].str.contains(r'\(|\)')

In [62]:
df.loc[paranthesis1, f"{last_fy} Totals"] = df.loc[paranthesis1, f"{last_fy} Totals"].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)
df[f"{last_fy} Totals"] = df[f"{last_fy} Totals"].str.replace('[\$,]', '', regex=True).astype(float)
df.loc[paranthesis2, f"{prev_prev_fy} Totals"] = df.loc[paranthesis2, f"{prev_prev_fy} Totals"].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)
df[f"{prev_prev_fy} Totals"] = df[f"{prev_prev_fy} Totals"].str.replace('[\$,]', '', regex=True).astype(float)

/var/folders/nc/1czn_dfn327cnn5jm9s4xv7w0000gn/T/ipykernel_49763/4180604813.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[paranthesis1, f"{last_fy} Totals"] = df.loc[paranthesis1, f"{last_fy} Totals"].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)
/var/folders/nc/1czn_dfn327cnn5jm9s4xv7w0000gn/T/ipykernel_49763/4180604813.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[paranthesis2, f"{prev_prev_fy} Totals"] = df.loc[paranthesis2, f"{prev_prev_fy} Totals"].str.replace(r'\(', '-').str.replace(r'\)', '', regex=True)


In [63]:
# df.head()

In [64]:
# df = df.reset_index(drop=True)
# print(df.iloc[7:12])

In [65]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)


In [66]:
# All modifications of dataframe
df.iloc[10, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES OPS'
df.iloc[45, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES ADMIN'
df.iloc[24, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES MAINT'
df = df.drop(columns=[f"Perc of {last_fy}", f"Perc of {prev_prev_fy}"], axis = 1)
# df = df.drop(columns=[f"Perc of {last_fy}", f"Perc of {prev_prev_fy}"], axis = 1)
# df['Percent Diff'] = np.where(df[f"{prev_prev_fy} Totals"] != 0, round(((df[f"{last_fy} Totals"] - df[f"{prev_prev_fy} Totals"])/df[f"{prev_prev_fy} Totals"])*100, 2), 100)

In [67]:
# print(df.columns)

In [68]:
df['Percent Diff'] = np.where(df[f"{prev_prev_fy} Totals"] != 0, round(((df[f"{last_fy} Totals"] - df[f"{prev_prev_fy} Totals"])/df[f"{prev_prev_fy} Totals"])*100, 2), 100)

In [69]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 6,
#                        ):
#     print(df)

#### CALCULATIONS OF CPOR

In [70]:
# columns for cost: totalcost of revenue, officer/admin salaries, payroll taxes, commercial ins, telephone, internet, property tax, 
# credit card commission, maint salary, payroll tax, pest, eqpt rep, fire and safety, swimming pool, maint&serv, plumbing, elec, trash&waste, 
# maint supp, total utilities, auto/gen insurance, building ins, 
rows_to_sum = ['Total Cost Of Revenue', 'OFFICERS SALARIES', 'ADMINISTRATIVE SALARIES', 'PAYROLL TAXES ADMIN', 'COMMERCIAL INSURANCE', 
               'TELEPHONE', 'INTERNET & COMPUTER', 'PROPERTY TAXES', 'CREDIT CARD COMMISSIONS', 'MAINTENANCE SALARIES', 'PAYROLL TAXES MAINT',
               'PEST CONTROL', 'EQUIPMENT REPAIR-GENERAL', 'FIRE & SAFETY', 'SWIMMING POOL', 'MAINTENANCE & SERV AGRMT', 'PLUMBING', 
               'ELECTRICAL & MECHANICAL', 'TRASH & WASTE REMOVAL', 'MAINTENANCE SUPPLIES', 'Total Utilities Expense', 'AUTO INSURANCE', 
               'GENERAL INSURANCE', 'BUILDING INSURANCE']
df['metric_type'] = df['metric_type'].str.lower().str.strip()
rows_to_sum = [row.lower().strip() for row in rows_to_sum]
temp = df['metric_type'].isin(rows_to_sum)
selectRows = df[temp]
totalOpCostCurrYr = selectRows[f"{last_fy} Totals"].sum()
totalOpCostPrevYr = selectRows[f"{prev_prev_fy} Totals"].sum()

In [18]:
roomsSoldCurrYr = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: 2022 total rooms = 35109
cPORCurrYr = totalOpCostCurrYr/int(roomsSoldCurrYr)
print(f"{last_fy}: ${round(cPORCurrYr, 2)}")

How many rooms sold? 35109


2022: $85.94


In [19]:
roomsSoldPrevYr = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: 2021 total rooms = 34811
cPORPrevYr = totalOpCostPrevYr/int(roomsSoldPrevYr)
print(f"{prev_prev_fy}: ${round(cPORPrevYr, 2)}")

How many rooms sold? 34811


2021: $65.01


In [20]:
print(selectRows)

                 metric_type  2022 Totals  2021 Totals  Percent Diff
20     total cost of revenue   1642520.57   1188779.98         38.17
22         officers salaries    260000.00    208416.52         24.75
23   administrative salaries    243685.44    193658.00         25.83
24       payroll taxes admin     39756.85     31892.43         24.66
25      commercial insurance     30740.68      1000.00       2974.07
31                 telephone     16172.54      6245.76        158.94
32       internet & computer     87073.62     71894.79         21.11
33   credit card commissions    120497.72     75523.21         59.55
40            property taxes     80048.70     87996.48         -9.03
44      maintenance salaries     52581.12     43806.80         20.03
45       payroll taxes maint     40365.24     30408.48         32.74
46              pest control     10347.70      2486.85        316.10
47  equipment repair-general     26992.05      2986.42        803.83
48             fire & safety     1

#### DISPLAY PERC OF TOTAL COLUMN FOR EXPENSE INSIGHT

In [21]:
totalCostRev = df.iloc[20][f"{last_fy} Totals"]
totalAdminRev = df.iloc[43][f"{last_fy} Totals"]
totalMaintRev = df.iloc[62][f"{last_fy} Totals"]

In [22]:
#to calculate perc of total column based off of total cost rev, total admin rev and total maint rev
df['Perc of Total'] = 0
df.loc[4:19, 'Perc of Total'] = (df.loc[4:20, f"{last_fy} Totals"]/totalCostRev)*100
df.loc[22:42, 'Perc of Total'] = (df.loc[22:42, f"{last_fy} Totals"]/totalAdminRev)*100
df.loc[44:61, 'Perc of Total'] = (df.loc[44:61, f"{last_fy} Totals"]/totalMaintRev)*100
print(df.loc[4:63])

                       metric_type  2022 Totals  2021 Totals  Percent Diff  \
4                 front desk clerk    164188.66    140538.56         16.83   
5                      housekeeper    474609.95    350408.83         35.44   
6                  café attendants      1003.00      4190.00        -76.06   
7                payroll taxes ops     18203.40     16550.54          9.99   
8                 pension expenses    284463.63    169969.75         67.36   
9           workers comp insurance     81203.21     84179.76         -3.54   
10                  linen purchase     18338.80      2657.52        590.07   
11                  guest supplies     71275.03     22951.31        210.55   
12              guest dry cleaning        29.50         0.00        100.00   
13                        uniforms         0.00       613.75       -100.00   
14                laundry supplies     17622.11     10352.86         70.21   
15        commission travel agents    206358.92    175975.70    

In [28]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [43]:
#visualization of perc of total column to view the pcts
costSlice = df.iloc[4:20]
adminSlice = df.iloc[22:43]
maintSlice = df.iloc[44:62]

titles = ['Cost Perc Total', 'Admin Perc Total', 'Maint Perc Total']

fig = make_subplots(rows=1,cols=3, subplot_titles=titles, specs=[[{"type":"domain"} for _ in range(3)] for _ in range(1)], print_grid=True)  
for i, data_slice in enumerate([costSlice, adminSlice, maintSlice]):
    label = data_slice['metric_type']
    size = data_slice['Perc of Total']
    fig_pie=go.Pie(values=size, labels=label, showlegend=False, textposition='inside', textinfo='percent', hoverinfo='label+percent')
    fig.add_trace(fig_pie, row=1, col=i+1)

fig.show()


This is the format of your plot grid:
[ (1,1)  ]  [ (1,2)  ]  [ (1,3)  ]



### Expense Ratio

In [45]:
df['expense_ratio'] = 0
df.loc[4:88, 'expense_ratio'] = df.loc[4:88, 'Percent Diff']/df.loc[3, 'Percent Diff']
print(df)

                   metric_type  2022 Totals  2021 Totals  Percent Diff  \
0          meeting room rental     69576.03     19399.03        258.66   
1                  room rental   4819339.23   4499654.34          7.10   
2                       refund     -4232.17    -32169.78        -86.84   
3                total revenue   4884683.09   4486883.59          8.87   
4             front desk clerk    164188.66    140538.56         16.83   
..                         ...          ...          ...           ...   
83                     penalty      3363.98       885.80        279.77   
84  provision for in tax-state      8990.00     13366.00        -32.74   
85    provision for in tax-pte     55738.00     93092.00        -40.13   
86          total other income  -1122836.17   -373270.46        200.81   
87           net income (loss)    390201.12   1462550.10        -73.32   

    Perc of Total  expense_ratio  
0         0.00000       0.000000  
1         0.00000       0.000000  
2     

In [57]:
cost_ops = df[4:20].copy()
admin_exp = df[22:43].copy()
maint_exp = df[44:62].copy()
ads_exp = df[63:65].copy()
util_exp = df[66:69].copy()

cost_ops['category'] = 'Cost Ops'
admin_exp['category'] = 'Admin Expense'
maint_exp['category'] = 'Maint Expense'
ads_exp['category'] = 'Ads Expense'
util_exp['category'] = 'Util Expense'

combined_df = pd.concat([cost_ops, admin_exp, maint_exp, ads_exp, util_exp], ignore_index=True)

bar = px.bar(combined_df, x='metric_type', y='expense_ratio', color = 'category', labels={'Expense Efficiency Ratio': 'Expense Efficiency Ratio'})
bar.update_yaxes(type='log')
bar.update_layout(
    title='Expense Efficiency Ratio by Metric Type',
    xaxis_title='Metric Type',
    yaxis_title='Expense Efficiency Ratio'
)
bar.add_annotation(
    text="Values > 1 indicate expense grew YoY compared to revenue for this metric",
    xref="paper", yref="paper",
    x=1, y=1.1,
    showarrow=False,
    font=dict(size=12)
)
bar.show()                                                             